# Langchain crash course

In [1]:
from secret_key import openapi_key
# import os
# os.environ['OPENAI_API_KEY'] = openapi_key

import os
os.environ['OPENAI_API_KEY'] = openapi_key

## LLMs

In [8]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.6)
name = llm.predict("I want to open a restaurant for Indian food. Suggest exactly one fency name for this.")
print(name)



"Saffron Spice Bistro"


In [9]:
llm("I want to open a restaurant for Indian food. Suggest a fency name for this.")

'\n\n1. "Spice Palace"\n2. "Taste of India"\n3. "Curry Mansion"\n4. "Saffron Delight"\n5. "Naan & Beyond"\n6. "Mughal Bites"\n7. "Royal Tandoor"\n8. "Flavors of the East"\n9. "The Indian Kitchen"\n10. "Masala Mansion"'

## Prompt Templates

In [11]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)
p = prompt_template_name.format(cuisine="Italian")
print(p)

print(llm.predict(p))

I want to open a restaurant for Italian food. Suggest a fency name for this.


"Bella Cucina" 


## Chains

In [12]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.run("Mexican")

C:\Users\kavin\AppData\Local\Temp\ipykernel_6692\723744833.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template_name)
C:\Users\kavin\AppData\Local\Temp\ipykernel_6692\723744833.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run("Mexican")


'\n\n"El Sabor Exquisito" (The Exquisite Flavor)'

In [13]:
chain = LLMChain(llm=llm, prompt=prompt_template_name, verbose=True)
chain.run("Mexican")



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for Mexican food. Suggest a fency name for this.

> Finished chain.


'\n\n"El Sabor Exquisito" (The Exquisite Flavor)'

In [14]:
llm = OpenAI(temperature=0.6)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="""Suggest some menu items for {restaurant_name}"""
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

#### Simple Sequential Chain

In [15]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])

content = chain.run("Indian")
print(content)



1. Spicy Chicken Tikka Masala
2. Lamb Vindaloo
3. Vegetable Korma
4. Tandoori Shrimp
5. Chana Masala
6. Samosas (vegetable or chicken)
7. Garlic Naan
8. Mango Lassi
9. Palak Paneer
10. Aloo Gobi
11. Butter Chicken
12. Biryani (vegetable, chicken, or lamb)
13. Dal Makhani
14. Tandoori Chicken
15. Gulab Jamun (dessert)


#### Sequential Chain

In [17]:
llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")
# output key =  to differntiate betweeen outputs of one chain.

In [18]:
llm = OpenAI(temperature=0.7)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

food_items_chain =LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

In [19]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', "menu_items"]
)

In [20]:
chain({"cuisine": "Indian"})

C:\Users\kavin\AppData\Local\Temp\ipykernel_6692\3052807682.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain({"cuisine": "Indian"})


{'cuisine': 'Indian',
 'restaurant_name': '\n\n"Taj Mahal Bistro"',
 'menu_items': '\n\n1. Chicken Tikka Masala\n2. Lamb Biryani\n3. Vegetable Samosas\n4. Tandoori Chicken\n5. Palak Paneer\n6. Butter Naan\n7. Malai Kofta\n8. Dal Makhani\n9. Aloo Gobi\n10. Rogan Josh\n11. Prawn Curry\n12. Mango Lassi\n13. Gulab Jamun\n14. Chicken Vindaloo\n15. Chana Masala\n16. Chicken Korma\n17. Matar Paneer\n18. Vegetable Biryani\n19. Garlic Naan\n20. Rasmalai.'}

## Agents

In [14]:
# make sure yoy have installed this package: pip install google-search-results
# from secret_key import serpapi_key
# os.environ['SERPAPI_API_KEY'] = serpapi_key

os.environ['SERPAPI_API_KEY'] = "add your serpapi key here"

#### serpapi and llm-math tool

In [21]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Let's test it out!
agent.run("What was the GDP of US in 2022 plus 5?")

ValidationError: 1 validation error for SerpAPIWrapper
  Value error, Did not find serpapi_api_key, please add an environment variable `SERPAPI_API_KEY` which contains it, or pass `serpapi_api_key` as a named parameter. [type=value_error, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

#### Wikipedia and llm-math tool

In [24]:
# install this package: pip install wikipedia

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True
)

# Let's test it out!
agent.run("When was Elon musk born? What is his age right now in 2023?")

C:\Users\kavin\AppData\Local\Temp\ipykernel_6692\1997441583.py:7: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(




> Entering new AgentExecutor chain...
 I should use Wikipedia to find information about Elon Musk's birthdate and age.
Action: wikipedia
Action Input: Elon Musk
Observation: Page: Elon Musk
Summary: Elon Reeve Musk ( EE-lon; born June 28, 1971) is a businessman known for his leadership of Tesla, SpaceX, and X (formerly Twitter). Since 2025, he has been a senior advisor to United States president Donald Trump and the de facto head of the Department of Government Efficiency (DOGE). Musk has been considered the wealthiest person in the world since 2021; as of May 2025, Forbes estimates his net worth to be US$424.7 billion. He was named Time magazine's Person of the Year in 2021.
Born to a wealthy family in Pretoria, South Africa, Musk emigrated in 1989 to Canada. He graduated from the University of Pennsylvania in the U.S. before moving to California to pursue business ventures. In 1995, Musk co-founded the software company Zip2. Following its sale in 1999, he co-founded X.com, an onlin

'Elon Musk will be 52 years old in 2023.'

## Memory

In [17]:
chain = LLMChain(llm=llm,prompt=prompt_template_name)
name = chain.run("Mexican")
print(name)



Taco Fiesta


In [18]:
name = chain.run("Indian")
print(name)



Maharaja's Palace Cuisine


In [25]:
chain.memory

In [26]:
type(chain.memory)

NoneType

#### ConversationBufferMemory

In [27]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name = chain.run("Mexican")
print(name)

C:\Users\kavin\AppData\Local\Temp\ipykernel_6692\3140656249.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()




"El Sabor Exquisito" (The Exquisite Flavor)


In [28]:
name = chain.run("Arabic")
print(name)



"Al-Fawarish" (meaning "The Treasures" in Arabic)


In [29]:
print(chain.memory.buffer)

Human: Mexican
AI: 

"El Sabor Exquisito" (The Exquisite Flavor)
Human: Arabic
AI: 

"Al-Fawarish" (meaning "The Treasures" in Arabic)


#### ConversationChain

In [30]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


C:\Users\kavin\AppData\Local\Temp\ipykernel_6692\3006166724.py:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  convo = ConversationChain(llm=OpenAI(temperature=0.7))


In [31]:
convo.run("Who won the first cricket world cup?")

' The first cricket world cup was held in 1975 in England and was won by the West Indies team. They defeated the Australian team by 17 runs in the final match. The West Indies team was captained by Clive Lloyd and had notable players such as Viv Richards, Gordon Greenidge, and Andy Roberts.'

In [32]:
convo.run("How much is 5+5?")

' 5+5 is equal to 10.'

In [33]:
convo.run("Who was the captain ofthe winning team?")

' As I mentioned earlier, the winning team was captained by Clive Lloyd. He was a talented batsman and an effective leader for the West Indies team.'

In [35]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup?
AI:  The first cricket world cup was held in 1975 in England and was won by the West Indies team. They defeated the Australian team by 17 runs in the final match. The West Indies team was captained by Clive Lloyd and had notable players such as Viv Richards, Gordon Greenidge, and Andy Roberts.
Human: How much is 5+5?
AI:  5+5 is equal to 10.
Human: Who was the captain ofthe winning team?
AI:  As I mentioned earlier, the winning team was captained by Clive Lloyd. He was a talented batsman and an effective leader for the West Indies team.


#### ConversationBufferWindowMemory

In [36]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)

convo = ConversationChain(
    llm=OpenAI(temperature=0.7),
    memory=memory
)
convo.run("Who won the first cricket world cup?")

C:\Users\kavin\AppData\Local\Temp\ipykernel_6692\2140498384.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=1)


" The first cricket world cup was won by the West Indies in 1975. They beat Australia by 17 runs in the final match, which was held at Lord's Cricket Ground in London. The West Indies team was led by Clive Lloyd, and they had players like Viv Richards, Gordon Greenidge, and Michael Holding on their roster. It was a historic win for the West Indies, as it was the first time they had won the world cup."

In [31]:
convo.run("How much is 5+5?")

' 5+5 is 10.'

In [32]:
convo.run("Who was the captain of the winning team?")

" I'm sorry, I don't know."